<img src="logo.png">

Los clasificadores Naive Bayes (Naive Bayes Classifier - NBC) se usan , como su nombre indica, para problemas de clasificación, y en concreto se pueden aplicar para texto. 

En este ejemplo vamos a implementar un modelo NBC a un dataset de un portal de Noticias muy famoso en España. Cada usuario comparte un link a una noticia y le puede asignar una categoría.

In [ ]:
import pandas as pd

noticias = pd.read_csv("noticias.csv").iloc[:2000,]

noticias.shape

La variable objetivo es `categoria` y la variable independiente es `descripcion` que contiene la descripcion de la noticia

In [ ]:
noticias.categoria.value_counts()

Vemos que hay noticias de 3 tipos de categorías distintas.

Los clasificadores Naive Bayes esperan como input un vector, así que para poder entrenarlos tenemos que vectorizar el texto. Para ello una buena opción es usar vectorización Tf-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
TfidfVectorizer?

Vamos a modificar nuestro vectorizador añadiendole dos parámetros:

** Eliminar Stopwords**

Stopwords (palabras vacías) son palabras que no tienen ningún contenido semántico. Por ejemplo, en la frase `el perro ladra` el artículo `el` no aporta ningún valor a la frase.


Me he descargado una lista de stopwords de castellano de este [repositorio en Github](https://github.com/stopwords-iso/stopwords-es).

In [ ]:
import json


with open("stopwords-es.json") as fname:
    stopwords_es = json.load(fname)

In [ ]:
stopwords_es[:30]

**Eliminar acentos**

También vamos a eliminar los acentos de las palabras, ésto tiene una ventaja, y es que si en el conjunto de datos no tenemos confianza en la calidad de los escritores, al eliminar los acentos evitamos que si un escritor no usa acentos no considere sus palabras como palabras distintas.

En castellano, esto tiene un problema, y es que hay palabras con significado distinto que sólo se diferencian por la existencia de una tilde (se llaman palabras con [acento diacrítico](https://es.wikipedia.org/wiki/Acento_diacr%C3%ADtico) (por ejemplo, `de y dé`). Asumimos pues que el impacto de estas palabras no es muy alto.

In [ ]:
vectorizador = TfidfVectorizer(strip_accents="unicode", stop_words=stopwords_es)

In [ ]:
noticias.shape

In [ ]:
vectorizador.fit_transform(noticias.descripcion)

Dicha matriz nos indica que tenemos 16495 artículos que tienen 59969 palabras distintas (sin contar acentos o stopwords)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

Dado que los vectorizadores devuelven una matriz sparse (escasa) creamos un transformador que las convierta a densas

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer

from scipy.sparse import issparse


# http://rasbt.github.io/mlxtend/
class DenseTransformer(BaseEstimator):
    def __init__(self, return_copy=True):
        self.return_copy = return_copy
        self.is_fitted = False

    def transform(self, X, y=None):
        if issparse(X):
            return X.toarray()
        elif self.return_copy:
            return X.copy()
        else:
            return X

    def fit(self, X, y=None):
        self.is_fitted = True
        return self

    def fit_transform(self, X, y=None):
        return self.transform(X=X, y=y)

In [ ]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

Scikit-learn tiene tres implementaciones del clasificador [Naive Bayes](http://scikit-learn.org/stable/modules/naive_bayes.html), `GaussianNB, BernoulliNB y MultinomialNB`, y cada una se diferencia por como calcula las probabilidades de que cada elemento aparezca en los datos.

[GaussianNB](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html) asume que los datos siguen una distribución Gausiana

In [ ]:
pipeline_gaussiano = make_pipeline(
    vectorizador,
    DenseTransformer(),
    GaussianNB()
)

In [ ]:
pipeline_gaussiano.fit(X=noticias.descripcion, y=noticias.categoria)

In [ ]:
pipeline_gaussiano.predict(noticias.descripcion)

In [ ]:
cross_val_score(pipeline_gaussiano, noticias.descripcion, noticias.categoria, scoring="f1")

In [ ]:
from sklearn.metrics import f1_score


In [ ]:
f1_score??

Vemos que la validación cruzada con la puntuación F1 nos da un error `ValueError: Target is multiclass but average='binary'. Please choose another average setting.`, esto es por que las medidas de clasificación tienen distintas maneras de calcularse en funcion de si es un caso de clasificación binaria (el parámetro por defecto) o clasificación multiclase. 

En concreto para el caso de la puntuación F1, nos permite los siguientes tipos de cálculos.

- **binary**: Devuelve la puntuación para la clase especificada en el argumento `pos_label`. Solo se puede aplicar en clasificación binaria. Este es el caso que vimos en el apartado de medidas de evaluación de modelos de clasificación.

- **micro**: Cuenta todos el número total de Verdaderos positivos (TP), Falsos Negativos (FN) y Falsos Positivos (FP) y calcula una precisión y sensibilidad total y obtiene el F1. Es mejor cuando hay clases no balanceadas (muchos casos más de una clase que de las demás).

- **macro**: Calcula la precisión y sensibilidad media de cada clase, hace su media aritmética y calcula el parámetro F1.

- **weighted**: Calcula la precisión y sensibilidad media de cada clase, hace su media ponderada por el número de observaciones de cada clase y calcula el parámetro F1.

- **samples**

In [ ]:
from sklearn.metrics import f1_score


def f1_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return f1_score(y, preds, average="micro")

In [ ]:
cross_val_score(pipeline_gaussiano, noticias.descripcion, noticias.categoria, scoring=f1_multietiqueta,cv=10)

Como hemos visto antes, el vectorizador TF-IDF coge por defecto todas las palabras (removiendo palabras sin aporte de información). Podemos restringir la cantidad de palabras que considera con el parámetro `max_features`.

In [ ]:
pipeline_gaussiano = make_pipeline(
    TfidfVectorizer(strip_accents="unicode", stop_words=stopwords_es, max_features=1000),
    DenseTransformer(),
    GaussianNB()
)

In [ ]:
cross_val_score(pipeline_gaussiano, noticias.descripcion, noticias.categoria, scoring=f1_multietiqueta)

Las dos implementaciones de clasificadores NB más utilizadas para clasificación de texto son [MultinomialNB](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB) (que asume que la distribución de probabilidades de las palabras en el conjunto de datos sigue una distribución multinomial y [BernouilliNB](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html#sklearn.naive_bayes.BernoulliNB) , que asume que siguen una distribución de Bernouilli multivariable (donde la existencia de cada palabra se considera que es una variable binaria distinta).

Según la documentación, el clasificador MultinomialNB funciona bien con vectores TFIDF

In [ ]:
pipeline_multinomial = make_pipeline(
    TfidfVectorizer(strip_accents="unicode", stop_words=stopwords_es, max_features=500),
    DenseTransformer(),
    MultinomialNB(),
)

In [ ]:
cross_val_score(pipeline_multinomial, noticias.descripcion, noticias.categoria,
                scoring=f1_multietiqueta)

Vemos que efectivamente, `MultinomialNB` parece funcionar mejor que `GaussianNB` para vectores TF-IDF

Para el clasificador [BernouilliNB](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html#sklearn.naive_bayes.BernoulliNB) se necesita tener los vectores de palabras cono vectores binarios (1 si la palabra existe o 0 si no), así que en vez de usar TfidfVectorizer en este caso usaremos el CountVectorizer pasandole el parámetro `binary=True` para que devuelva 1 ó 0 en vez del número de veces que aparece la palabra en la frase.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizador_count = CountVectorizer(stop_words=stopwords_es, binary=True, 
                                     strip_accents="unicode", max_features=1000)

Una funcionalidad interesante del CountVectorizer es que nos permite ver cuantas veces aparece cada palabra en el corpus.

In [ ]:
vectorizador_count.fit(noticias.descripcion)
vectorizador_count.vocabulary_

In [ ]:
pipeline_bernouilli = make_pipeline(
    vectorizador_count,
    DenseTransformer(),
    BernoulliNB(),
)

In [ ]:
cross_val_score(pipeline_bernouilli, noticias.descripcion, noticias.categoria, scoring=f1_multietiqueta)